In [1]:
import requests 
from bs4 import BeautifulSoup as bs
import pandas as pd
from tqdm import tqdm

In [2]:
# date는 2020.01.01 형식으로 받아옴 

def joongang_crawling(start_date, end_date):
    eng_url_1 = 'https://news.joins.com/Search/JoongangNews?Keyword=kaist'
    kor_url_1 = 'https://news.joins.com/Search/JoongangNews?Keyword=카이스트'
    url_2 = '&StartSearchDate='+str(start_date)+'&EndSearchDate='+str(end_date)
    url_3 = '&SortType=New&SearchCategoryType=JoongangNews&PeriodType=DirectInput&ScopeType=TitleContent&ImageType=All&JplusType=All&BlogType=All'
    url_4 = '&ImageSearchType=Image&TotalCountStartCount=0&IsChosung=False&IssueCategoryType=All&IsDuplicate=True&Page='
    url_5 = '&PageSize=10&IsNeedTotalCount=True'
    eng_url_head = eng_url_1 + url_2 + url_3 + url_4
    eng_url_tail = url_5
    kor_url_head = kor_url_1 + url_2 + url_3 + url_4
    kor_url_tail = url_5
    
    # 검색 결과에 따른 url 링크 저장
    url1 = []
    # url 변화하는거 적당히 보고 알아서 잘 해야됨. 페이지 단위로 넘어가는게 아니라면 selenium 써야할 수도 있음.
    page = 1
    while True:
        url_final = eng_url_head + str(page) + eng_url_tail
        try:
            response = requests.get(url_final)
            html = response.content
            soup = bs(html, "html.parser")
#            page가 넘어갔을 때 검색결과가 없는 경우 while 문 break
            if len(soup.select('.section_no_result')) != 0:
                break
            elements = soup.select('.thumb a[href]') # F12 눌러서 적당히 우리가 원하는 a href 태그 url 링크 달려있는 id 또는 class 찾아서 넣어주면 됨.
        except:
            print('Request Failed')
            break
        for element in elements:
          link = element.attrs['href']
          url1.append(link)
        page += 1
            
    # 검색 결과에 따른 url2 링크 저장
    url2 = []
    page = 1
    while True:
        url_final = kor_url_head + str(page) + kor_url_tail
        try:
            response = requests.get(url_final)
            html = response.content
            soup = bs(html, "html.parser")
#            page가 넘어갔을 때 검색결과가 없는 경우 while 문 break
            if len(soup.select('.section_no_result')) != 0:
                break
            elements = soup.select('.thumb a[href]') # F12 눌러서 적당히 우리가 원하는 a href 태그 url 링크 달려있는 id 또는 class 찾아서 넣어주면 됨.
        except:
            print('Request Failed')
            break
        for element in elements:
          link = element.attrs['href']
          url2.append(link)
        page += 1
    url = list(set(url1).union(set(url2)))
    if len(url)==0:
        print("Articles Not Found")
        return 0
    df = pd.DataFrame({"url": url}, columns=["url"])
    contents = []
    titles = []
    urls = []
    for row in tqdm(df.itertuples()):
        link = getattr(row, 'url')
        response = requests.get(link)
        html = response.text
        soup = bs(html,'html.parser')
        try:
            for script in soup(["script", "style"]):
                script.decompose() 
            content = soup.select('#article_body')[0].get_text() # select 안에 본문에 해당하는 id 값을 찾아서 넣어주면 됨.
            content = content.replace('\xa0', " ").replace('\u3000','').replace('\r','').replace('\n', " ")
            contents.append(content)
            title = soup.select('#article_title')[0].get_text() # select 안에 본문에 해당하는 id 값을 찾아서 넣어주면 됨.
            title = title.replace('\xa0', " ")
            title = title.replace('\n', " ")
            titles.append(title)
            urls.append(link)
        except:
            print(row)
    df = pd.DataFrame({"url": urls, "title": titles, "content": contents}, columns=["url", "title", "content"])
    df.to_excel("articles/joongang.xlsx",index = False)
    df.to_pickle("pickle files/joongang.pkl")

In [3]:
joongang_crawling('2015.01.01', '2020.10.23')

1959it [26:38,  1.23it/s]
